# Tableau

本手册提供一个快速入门指南，介绍如何使用 [Tableau](https://help.tableau.com/current/api/vizql-data-service/en-us/index.html)。

### 概述

Tableau 的 VizQL 数据服务（又名 VDS）为开发人员提供了对其 Tableau 已发布数据源的编程访问权限，使他们能够为任何自定义工作负载或应用程序（包括 AI Agent）扩展其业务语义。`simple_datasource_qa` 工具将 VDS 添加到 Langchain 框架。本笔记本展示了如何使用它来构建基于企业语义模型回答分析问题的代理（Agents）。

请关注 [tableau-langchain](https://github.com/Tab-SE/tableau_langchain) 项目，了解即将推出的更多工具！

#### 设置
请确保您正在运行并可以访问以下环境：
1. Python 3.12.2 或更高版本
2. Tableau Cloud 或 Server 环境，并且至少有一个已发布的数据源

通过安装和/或导入所需的包即可开始

In [ ]:
# %pip install langchain-openai

In [ ]:
# %pip install langgraph

In [ ]:
# %pip install langchain-tableau --upgrade

请注意，使用更新的包可能需要重新启动内核。

### 凭证

你可以显式声明环境变量，如本文档中的多个示例所示。但是，如果未提供这些参数，simple_datasource_qa 工具将尝试从环境变量中自动读取它们。

对于你选择查询的数据源，请确保已在 Tableau 中更新 VizqlDataApiAccess 权限，允许 VDS API 通过 REST 访问该数据源。更多信息请参见[此处](https://help.tableau.com/current/server/en-us/permissions_capabilities.htm#data-sources)。

In [1]:
# langchain package imports
from langchain_openai import ChatOpenAI

# langchain_tableau and langgraph imports
from langchain_tableau.tools.simple_datasource_qa import initialize_simple_datasource_qa
from langgraph.prebuilt import create_react_agent

## 认证变量

你可以像本手册中的几个例子那样，显式声明你的环境变量。但是，如果未提供这些参数，simple_datasource_qa 工具将尝试从环境变量中自动读取它们。

对于你选择的数据源，请确保你已更新 Tableau 中的 `VizqlDataApiAccess` 权限，允许 VDS API 通过 REST 访问该数据源。更多信息请参见[此处](https://help.tableau.com/current/server/en-us/permissions_capabilities.htm#data-sources
)。

In [ ]:
import os

from dotenv import load_dotenv

load_dotenv()

tableau_server = "https://stage-dataplane2.tableau.sfdc-shbmgi.svc.sfdcfc.net/"  # replace with your Tableau server name
tableau_site = "vizqldataservicestage02"  # replace with your Tableau site
tableau_jwt_client_id = os.getenv(
    "TABLEAU_JWT_CLIENT_ID"
)  # a JWT client ID (obtained through Tableau's admin UI)
tableau_jwt_secret_id = os.getenv(
    "TABLEAU_JWT_SECRET_ID"
)  # a JWT secret ID (obtained through Tableau's admin UI)
tableau_jwt_secret = os.getenv(
    "TABLEAU_JWT_SECRET"
)  # a JWT secret ID (obtained through Tableau's admin UI)
tableau_api_version = "3.21"  # the current Tableau REST API Version
tableau_user = "joe.constantino@salesforce.com"  # enter the username querying the target Tableau Data Source

# For this cookbook we are connecting to the Superstore dataset that comes by default with every Tableau server
datasource_luid = (
    "0965e61b-a072-43cf-994c-8c6cf526940d"  # the target data source for this Tool
)
model_provider = "openai"  # the name of the model provider you are using for your Agent
# Add variables to control LLM models for the Agent and Tools
os.environ["OPENAI_API_KEY"]  # set an your model API key as an environment variable
tooling_llm_model = "gpt-4o-mini"

## 实例化
`initialize_simple_datasource_qa` 初始化了 Langgraph 工具 [simple_datasource_qa](https://github.com/Tab-SE/tableau_langchain/blob/3ff9047414479cd55d797c18a78f834d57860761/pip_package/langchain_tableau/tools/simple_datasource_qa.py#L101)，该工具可用于 Tableau 数据源上的分析问答。

此初始化器函数：
1. 使用 Tableau 的 connected-app 框架（基于 JWT 的身份验证）向 Tableau 进行身份验证。所有必需的变量必须在运行时或作为环境变量定义。
2. 异步查询指定 `datasource_luid` 变量的目标数据源的字段元数据。
3. 利用目标数据源的元数据，将自然语言问题转换为 VDS 查询数据源请求所需的 JSON 格式查询负载。
4. 执行 POST 请求到 VDS。
5. 以结构化的响应格式化并返回结果。

In [6]:
# Initialize simple_datasource_qa for querying Tableau Datasources through VDS
analyze_datasource = initialize_simple_datasource_qa(
    domain=tableau_server,
    site=tableau_site,
    jwt_client_id=tableau_jwt_client_id,
    jwt_secret_id=tableau_jwt_secret_id,
    jwt_secret=tableau_jwt_secret,
    tableau_api_version=tableau_api_version,
    tableau_user=tableau_user,
    datasource_luid=datasource_luid,
    tooling_llm_model=tooling_llm_model,
    model_provider=model_provider,
)

# load the List of Tools to be used by the Agent. In this case we will just load our data source Q&A tool.
tools = [analyze_datasource]

## 调用 - Langgraph 示例
首先，我们将初始化我们选择的 LLM。然后，我们使用 langgraph 代理构造类定义一个代理，并使用与目标数据源相关的查询来调用它。

In [ ]:
from IPython.display import Markdown, display

model = ChatOpenAI(model="gpt-4o", temperature=0)

tableauAgent = create_react_agent(model, tools)

# Run the agent
messages = tableauAgent.invoke(
    {
        "messages": [
            (
                "human",
                "what's going on with table sales?",
            )
        ]
    }
)
messages
# display(Markdown(messages['messages'][3].content)) #display a nicely formatted answer for successful generations

## 链式调用

待办事项。

## API 参考

待办事项。